In [ ]:
import pandas as pd
import regex as re
import pickle
import gensim
import pyLDAvis
# from pyLDAvis import gensim

notes = pd.read_csv('notes-2.tsv', sep="\t")
ratings = pd.read_csv('ratings-2.tsv', sep="\t")
tweets = pd.read_csv('noted-tweets.csv', sep=",")

/shared-libs/python3.8/py-core/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
print(len(tweets), len(notes), len(ratings))

12708 16268 93778
/shared-libs/python3.8/py-core/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
with open('temp.pickle', 'rb') as handle:
    d = pickle.load(handle)
tweets = d['tweets']

In [ ]:
tweets.head()

,Unnamed: 0,tweetId,username,userId,text,time,link,links,clean_text,word_count,num_links,has_links,polarity,subjectivity,clean_text_as_str
1,1,1355643797280083973,rjnln02,1.629330e+09,@AF632 @alwsbord1 Black Lives Matter is a raci...,2021-01-30 22:27:22,https://twitter.com/rjnln02/status/13556437972...,[],"[af, alwsbord, black, life, matter, racist, do...",132,0,0,-0.083333,0.266667,af alwsbord black life matter racist domestic ...
2,2,1355529775197130753,luscsmoothie,3.075792e+08,@nobelprize To remind u what BLM did. Disquali...,2021-01-30 14:54:17,https://twitter.com/luscsmoothie/status/135552...,[https://t.co/2qDB0db13x],"[nobelprize, remind, u, blm, disqualify, rt, n...",119,1,1,0.000000,0.000000,nobelprize remind u blm disqualify rt nextlama...
3,3,1352754599590035459,DonaldJTrumpJr,3.934437e+07,Donald Trump is the first president in modern ...,2021-01-22 23:06:44,https://twitter.com/DonaldJTrumpJr/status/1352...,[],"[donald, trump, president, modern, history, st...",77,0,0,0.168182,0.377273,donald trump president modern history start ne...
4,4,1356441140959064064,FionaRamshackle,8.664043e+17,@dcexaminer Go get those Antifa bastards! And ...,2021-02-02 03:15:44,https://twitter.com/FionaRamshackle/status/135...,[],"[dcexaminer, antifa, bastard, random, anarchis...",284,0,0,-0.500000,0.500000,dcexaminer antifa bastard random anarchist boo...
5,5,1361810447431827459,DanCrenshawTX,9.305526e+17,This raises the obvious question: can we ever ...,2021-02-16 22:51:26,https://twitter.com/DanCrenshawTX/status/13618...,[],"[raise, obvious, question, rely, renewable, po...",231,0,0,0.003788,0.666667,raise obvious question rely renewable power gr...


In [ ]:
the import gensim.corpora as corpora

# Create Dictionary
dictionary = corpora.Dictionary(tweets['clean_text'])

# Term Document Frequency
corpus = [dictionary.doc2bow(text) for text in tweets['clean_text']]

# View
print(corpus[:1][0][:30])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1)]


In [ ]:
from pprint import pprint
num_topics = 10

# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=dictionary,
                                       num_topics=num_topics)

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.013*"biden" + 0.012*"president" + 0.012*"trump" + 0.008*"people" + '
  '0.008*"vote" + 0.007*"ivermectin" + 0.007*"not" + 0.007*"amp" + 0.007*"s" + '
  '0.007*"million"'),
 (1,
  '0.019*"not" + 0.013*"people" + 0.012*"know" + 0.012*"trump" + 0.008*"covid" '
  '+ 0.006*"win" + 0.006*"want" + 0.006*"vaccine" + 0.006*"biden" + 0.006*"s"'),
 (2,
  '0.055*"tweet" + 0.031*"birdwatch" + 0.030*"try" + 0.027*"add" + '
  '0.027*"context" + 0.025*"want" + 0.022*"mislead" + 0.022*"hit" + '
  '0.022*"open" + 0.021*"helpful"'),
 (3,
  '0.020*"earthquake" + 0.019*"likely" + 0.013*"people" + 0.011*"risk" + '
  '0.009*"san" + 0.008*"francisco" + 0.008*"covid" + 0.008*"california" + '
  '0.008*"not" + 0.007*"ivermectin"'),
 (4,
  '0.011*"not" + 0.006*"stop" + 0.006*"want" + 0.006*"hour" + 0.005*"trump" + '
  '0.005*"election" + 0.005*"french" + 0.005*"likely" + 0.005*"white" + '
  '0.004*"kill"'),
 (5,
  '0.016*"covid" + 0.015*"not" + 0.011*"people" + 0.011*"vaccine" + '
  '0.009*"mask" + 0.00

In [ ]:
!pip install pyLDAvis.gensim

/shared-libs/python3.8/py-core/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
ERROR: Could not find a version that satisfies the requirement pyLDAvis.gensim (from versions: none)
ERROR: No matching distribution found for pyLDAvis.gensim


In [ ]:
import pyLDAvis.gensim_models as gensimvis

/shared-libs/python3.8/py-core/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:

# import pyLDAvis.gensim

# Visualize the topics
pyLDAvis.enable_notebook()

filepath = 'ldavis_prepared_'+str(num_topics)

if True:
    LDAvis_prepared = gensimvis.prepare(lda_model, corpus, dictionary)
    with open(filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)

# load the pre-prepared pyLDAvis data from disk
with open(filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)

pyLDAvis.save_html(LDAvis_prepared, filepath +'.html')

LDAvis_prepared

/shared-libs/python3.8/py-core/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.048666 -0.061965       1        1  13.669354
2      0.223441  0.046006       2        1  11.223962
3     -0.101903  0.121210       3        1  10.690959
9     -0.044920 -0.074385       4        1  10.544984
6     -0.011265 -0.012237       5        1  10.372597
5     -0.081717  0.027112       6        1   9.899284
1      0.009247 -0.038721       7        1   9.087326
4     -0.006943  0.066138       8        1   8.916311
8      0.019223 -0.073829       9        1   8.471077
7      0.043503  0.000670      10        1   7.124145, topic_info=          Term         Freq        Total Category  logprob  loglift
111      tweet  1220.000000  1220.000000  Default  30.0000  30.0000
102  birdwatch   650.000000   650.000000  Default  29.0000  29.0000
103    context   565.000000   565.000000  Default  28.0000  28.0000
101        add   589.000000   589.000000  Default  27.0000  27.0000
110        try   776.000000   776.000000  Default  26.0000  26.0000
..         ...          ...          ...      ...      ...      ...
77        come    32.061643   288.651411  Topic10  -5.7951   0.4441
32       trump    35.233683   941.473637  Topic10  -5.7008  -0.6438
432        get    32.378142   348.183760  Topic10  -5.7853   0.2664
118       year    32.035169   376.475406  Topic10  -5.7959   0.1777
105    helpful    31.719375   456.499151  Topic10  -5.8058  -0.0250

[799 rows x 6 columns], token_table=       Topic      Freq    Term
term                          
1000       1  0.015319       _
1000       2  0.068937       _
1000       4  0.061277       _
1000       5  0.022979       _
1000       6  0.520859       _
...      ...       ...     ...
676       10  0.065297    york
10828     10  0.875509  yorker
5392       1  0.093288    zinc
5392       3  0.093288    zinc
5392       6  0.746303    zinc

[2514 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 4, 10, 7, 6, 2, 5, 9, 8])

## Unfinished Explorations

Suspiciously common vector analysis

In [ ]:
# nums = '0.5165612 -0.2748843 0.23414916 -0.03353653 0.14822118 -0.2140591 -0.273715 0.31182715 -0.28392965 0.03833102 0.71412915 -0.12623966 0.01060359 -0.22177511 -0.48231328 0.29029948 0.19768755 0.23107074 -0.6973527 -0.02526714 -0.32837042 0.12478355 0.01376466 -0.16914183 0.3283727 -0.45044652 0.17814918 -0.49533963 0.1848702 -0.14078267 0.21354735 0.33570722 -0.16036382 -0.24869299 0.11315513 -0.35369763 -0.1663535 0.5083657 -0.04987437 0.4141711 0.10442343 0.41257355 -0.41149414 0.26209757 0.26663744 -0.21748962 0.08647469 -0.06590172 -0.24378727 -0.27255812 0.5440032 -0.27095118 -0.1553473 0.0673851 -0.03265667 -0.22625995 0.43378648 -0.0917755 0.10038939 0.04093109 0.349573 -0.41145465 0.2382577 -0.36835834 0.02527652 -0.21686192 -0.7478197 0.04894651 -0.07372946 0.07308427 0.12849472 0.21438135 -0.24230045 -0.23281254 -0.04018355 0.17432278 0.01564372 -0.08756901 -0.4357647 0.07869332 0.11019602 0.4734359 0.3662394 0.40063536 -0.2054153 -0.12328431 -0.03628931 -0.60807943 0.04818559 0.3079439 -0.5035178 0.02922824 0.45156133 -0.10860304 -0.133015 0.27818713'

nums = '0.5165612 -0.2748843 0.23414916 -0.03353653 0.14822118 -0.2140591 -0.273715 0.31182715 -0.28392965 0.03833102 0.71412915 -0.12623966 0.01060359 -0.22177511 -0.48231328 0.29029948 0.19768755 0.23107074 -0.6973527 -0.02526714 -0.32837042 0.12478355 0.01376466 -0.16914183 0.3283727 -0.45044652 0.17814918 -0.49533963 0.1848702 -0.14078267 0.21354735 0.33570722 -0.16036382 -0.24869299 0.11315513 -0.35369763 -0.1663535 0.5083657 -0.04987437 0.4141711 0.10442343 0.41257355 -0.41149414 0.26209757 0.26663744 -0.21748962 0.08647469 -0.06590172 -0.24378727 -0.27255812 0.5440032 -0.27095118 -0.1553473 0.0673851 -0.03265667 -0.22625995 0.43378648 -0.0917755 0.10038939 0.04093109 0.349573 -0.41145465 0.2382577 -0.36835834 0.02527652 -0.21686192 -0.7478197 0.04894651 -0.07372946 0.07308427 0.12849472 0.21438135 -0.24230045 -0.23281254 -0.04018355 0.17432278 0.01564372 -0.08756901 -0.4357647 0.07869332 0.11019602 0.4734359 0.3662394 0.40063536 -0.2054153 -0.12328431 -0.03628931 -0.60807943 0.04818559 0.3079439 -0.5035178 0.02922824 0.45156133 -0.10860304 -0.133015 0.27818713'.split()
nums = [float(x) for x in nums]

In [ ]:
def my_helpful_vector_comparison(row):
    try:
        return np.allclose(row.vector, nums)
    except:
        return False

In [ ]:
suspicious_vector_mask = X_notes.apply(axis=1, func=my_helpful_vector_comparison)
suspicious_vector_mask

NameError: name 'X_notes' is not defined

In [ ]:
X_notes[suspicious_vector_mask]

In [ ]:
notes.merge(X_notes[suspicious_vector_mask], left_on='noteId', right_index=True)

Look at empty tweets.


In [ ]:
test = X_notes[X_notes['vector'].apply(lambda x: len(x) == 0)]

In [ ]:
X_notes['vector'].apply(lambda x: len(x)).value_counts()

In [ ]:
test.merge(notes, left_index=True, right_on='noteId')

In [ ]:
import pandas as pd
notes = pd.read_csv('notes-2.tsv', sep="\t")
ratings = pd.read_csv('ratings-2.tsv', sep="\t")

In [ ]:
ratings.head()


,noteId,participantId,createdAtMillis,version,agree,disagree,helpful,notHelpful,helpfulnessLevel,helpfulOther,...,notHelpfulIncorrect,notHelpfulSourcesMissingOrUnreliable,notHelpfulOpinionSpeculationOrBias,notHelpfulMissingKeyPoints,notHelpfulOutdated,notHelpfulHardToUnderstand,notHelpfulArgumentativeOrInflammatory,notHelpfulOffTopic,notHelpfulSpamHarassmentOrAbuse,notHelpfulIrrelevantSources
0,1352796878438424576,7585B8804A32416E91E51837F351F249,1611388222120,1,1,0,1,0,NaN,0,...,0,0,0,0,0,0,0,0,0,0
1,1352796878438424576,7644DF3FD853416F0C96933CCC1BA9B7,1611796572477,1,0,1,0,1,NaN,0,...,0,0,0,0,0,0,0,0,1,0
2,1352796878438424576,EFD7E04E740224D2DDB42A2C910B62C1,1611852744990,1,1,0,1,0,NaN,0,...,0,0,0,0,0,0,0,0,0,0
3,1352796878438424576,3C9A15EEA8FD238754BE28C15B26709A,1611673955881,1,1,0,0,1,NaN,0,...,0,1,0,0,0,0,0,0,0,0
4,1352796878438424576,0466BA47B23AAAB301AA767C4C780E8D,1611523319844,1,1,0,1,0,NaN,0,...,0,0,0,0,0,0,0,0,0,0


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=c0e62a2c-7f45-414e-8164-5bf51e09d482' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>